# import libraries

In [1]:
import numpy as np
import pandas as pd
import pymysql
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix
import matplotlib.pyplot as plt
from drive_and_win_bib import *

# Data preprocessing

### fetch the database

In [2]:
try:
#fetch the data from the serveur   
    print("tentative d'access")
    cnx=pymysql.connect(host='51.77.149.46',user='djagora',passwd='djagora123@student1',db='djagora_student1',charset='utf8mb4',cursorclass=pymysql.cursors.DictCursor, read_timeout=30)
    cursor = cnx.cursor()
    print(cursor)
    data=pd.read_sql_query("SELECT * FROM data",cnx)
    print("connexion to database successful")
except:
#if serveur connection failed fetch a local version of the database
    print("connexion to local database")
    data = pd.read_csv('./data01122020.csv', sep=',',header=0)
    data = pd.DataFrame(data)
    data.columns=['CIN', 'altitude', 'longitude', 'Speed','RPM','ENGINE_LOAD','AmbientAirTemp','ThrottlePos', 'insFuel','X','Y','Z','zone','time']
data.head(10)

tentative d'access
connexion to local database


,CIN,altitude,longitude,Speed,RPM,ENGINE_LOAD,AmbientAirTemp,ThrottlePos,insFuel,X,Y,Z,zone,time
0,6023226,35.081887,9.869902,20.0,2810,"40,4%",36C,"23,9%",NaN,NaN,NaN,NaN,Nationale,2020-07-05 21:13:12
1,6023226,35.081886,9.870205,26.0,2116,"83,5%",36C,"33,7%",NaN,NaN,NaN,NaN,Nationale,2020-07-05 21:13:16
2,6023226,35.081931,9.870459,37.0,1828,"64,7%",36C,"25,9%",NaN,NaN,NaN,NaN,Nationale,2020-07-05 21:13:19
3,6023226,35.081950,9.871037,41.0,2019,"64,7%",36C,"26,3%",NaN,NaN,NaN,NaN,Nationale,2020-07-05 21:13:22
4,6023226,35.081988,9.871456,44.0,2173,"61,6%",36C,"27,5%",NaN,NaN,NaN,NaN,Nationale,2020-07-05 21:13:26
5,6023226,35.082021,9.871924,42.0,2035,"1,6%",36C,"16,5%",NaN,NaN,NaN,NaN,Nationale,2020-07-05 21:13:29
6,6023226,35.082140,9.872353,36.0,1805,"94,1%",36C,"88,2%",NaN,NaN,NaN,NaN,Nationale,2020-07-05 21:13:34
7,6023226,35.082202,9.872727,43.0,2049,"12,9%",36C,"16,5%",NaN,NaN,NaN,NaN,Nationale,2020-07-05 21:13:37
8,6023226,35.082236,9.873162,35.0,1706,"0,0%",36C,"16,1%",NaN,NaN,NaN,NaN,Nationale,2020-07-05 21:13:40
9,6023226,35.082291,9.873536,31.0,1495,"9,8%",36C,"15,7%",NaN,NaN,NaN,NaN,Nationale,2020-07-05 21:13:44


### Clean database from comments

In [3]:
data = data.drop(['insFuel'],axis=1)
data = datacleaning_comment (data)

### Transform database to numbers

In [4]:
def convert_to_number (data):
    data['time'] =  pd.to_datetime(data['time'])
    data = data.convert_dtypes(convert_string=False)
    
    dataconv =  data.apply(pd.to_numeric ,errors="coerce")
    dataconv = dataconv.drop(['zone','time'],axis=1)
    dataconv['zone'] = data['zone']
    dataconv['time'] = data['time']
    data = dataconv
    print(data['zone'])
    return(data)


In [5]:
data = convert_to_number(data)
data.head(5)

0        Nationale
1        Nationale
2        Nationale
3        Nationale
4        Nationale
           ...    
42203      Urbaine
42204      Urbaine
42205      Urbaine
42206      Urbaine
42207      Urbaine
Name: zone, Length: 42208, dtype: object


,CIN,altitude,longitude,Speed,RPM,ENGINE_LOAD,AmbientAirTemp,ThrottlePos,X,Y,Z,zone,time
0,6023226,35.081887,9.869902,20,2810.0,40.4,36.0,23.9,NaN,NaN,NaN,Nationale,2020-07-05 21:13:12
1,6023226,35.081886,9.870205,26,2116.0,83.5,36.0,33.7,NaN,NaN,NaN,Nationale,2020-07-05 21:13:16
2,6023226,35.081931,9.870459,37,1828.0,64.7,36.0,25.9,NaN,NaN,NaN,Nationale,2020-07-05 21:13:19
3,6023226,35.081950,9.871037,41,2019.0,64.7,36.0,26.3,NaN,NaN,NaN,Nationale,2020-07-05 21:13:22
4,6023226,35.081988,9.871456,44,2173.0,61.6,36.0,27.5,NaN,NaN,NaN,Nationale,2020-07-05 21:13:26


### Drop Nan values from Accelerometer

In [6]:
try:
    data = data.dropna(subset = ['valX', 'valY', 'valZ']) # if connected to distant database
except:
    data=data.dropna(subset = ['X', 'Y', 'Z'])# if connected tolocal database
data = data.reset_index(drop=True)

In [7]:
data.head(3)

,CIN,altitude,longitude,Speed,RPM,ENGINE_LOAD,AmbientAirTemp,ThrottlePos,X,Y,Z,zone,time
0,6023226,34.729739,10.655240,<NA>,NaN,NaN,NaN,NaN,1.610234,8.581922,5.403137,Urbaine,2020-07-22 17:12:12
1,6023226,34.729750,10.655937,<NA>,NaN,NaN,NaN,NaN,2.545130,3.656662,8.467283,Urbaine,2020-07-22 17:12:16
2,6023226,34.729785,10.656346,43,2112.0,72.2,42.0,29.0,0.083398,7.703288,5.959764,Urbaine,2020-07-22 17:12:20


In [8]:
data.shape

(38710, 13)

### Create a function that transforms the accelerometer values to angle of rotation

### Create angle rotation column

In [12]:
import math
angle_list,angle1, angle2, diff = [],0,0,0
for i in range (len(data)):
    if(i < len(data)-2): 
        angle1 = float(getBearing( float(data['altitude'].iloc[i]) , float(data['altitude'].iloc[i+1]), float(data['longitude'].iloc[i]),  float(data['longitude'].iloc[i+1])))
        angle2 = float(getBearing( float(data['altitude'].iloc[i+1]),float(data['altitude'].iloc[i+2]), float(data['longitude'].iloc[i+1]),float(data['longitude'].iloc[i+2])))
        diff = float( max(angle1, angle2) - min(angle1, angle2))
        if (diff > 180):
            diff = 360 - diff
        angle_list.append(float(diff))
angle_list.extend([None for i in range (len(data)-len(angle_list))])

data['Angle_Rot']= np.array(angle_list)

In [10]:
xacc = data['X'] / 9.8 #- data['valX'].mean()
yacc = data['Y'] /9.8  #- data['valY'].mean()
zacc = data['Z'] / 9.8 #- data['valZ'].mean()
xacc2 = (xacc*xacc)
yacc2 = (yacc*yacc)
zacc2 = (zacc*zacc)
acc_angX = np.arctan(xacc /np.sqrt(yacc2+zacc2)) *57.32 #(180 / pi)
acc_angY = np.arctan(yacc /np.sqrt(xacc2+zacc2))*57.32 # (180 / pi)
acc_angZ = np.arctan(zacc /np.sqrt(xacc2+yacc2)) *57.32 # (180 / pi)
data['acc_angX'] = acc_angX
data['acc_angY'] = acc_angY
data['acc_angZ'] = acc_angZ

In [13]:
data['Angle_Rot']

0         4.07295
1         5.83896
2         4.34468
3         1.12742
4         6.06838
           ...   
38705    0.782921
38706     11.0593
38707     28.8805
38708        None
38709        None
Name: Angle_Rot, Length: 38710, dtype: object

### extracting columns that have missing values

In [14]:
data.count()

CIN               38710
altitude          38710
longitude         38710
Speed             34943
RPM               34940
ENGINE_LOAD       34788
AmbientAirTemp    34904
ThrottlePos       34783
X                 38710
Y                 38710
Z                 38710
zone              38710
time              38710
acc_angX          38710
acc_angY          38710
acc_angZ          38710
Angle_Rot         38708
dtype: int64

In [15]:
missing_values_list = []
for i in range (len(data.columns)):
    if data.count()[i] != data['CIN'].count():
        missing_values_list.append(data.count().index[i])
data_missing_v = data[missing_values_list]

In [16]:
data_missing_v.count()

Speed             34943
RPM               34940
ENGINE_LOAD       34788
AmbientAirTemp    34904
ThrottlePos       34783
Angle_Rot         38708
dtype: int64

In [17]:
data_missing_v

,Speed,RPM,ENGINE_LOAD,AmbientAirTemp,ThrottlePos,Angle_Rot
0,<NA>,NaN,NaN,NaN,NaN,4.07295
1,<NA>,NaN,NaN,NaN,NaN,5.83896
2,43,2112.0,72.2,42.0,29.0,4.34468
3,52,2559.0,76.9,42.0,30.6,1.12742
4,61,2974.0,78.0,42.0,34.9,6.06838
...,...,...,...,...,...,...
38705,25,2027.0,NaN,35.0,NaN,0.782921
38706,28,2204.0,22.7,35.0,18.4,11.0593
38707,19,1500.0,0.0,35.0,14.9,28.8805
38708,19,1503.0,29.0,36.0,17.3,None


### interpolate on the data with the missing values

In [18]:
def completing_missing_data(data):
    try:
        data_missing_v= data_missing_v.interpolate(method="cubic",limit=3)
    except:
        data['Speed'] = data['Speed'].astype('float')
        data_missing_v = data[missing_values_list]
        data_missing_v= data_missing_v.interpolate(method="cubic",limit=3)
    return (data_missing_v)

In [19]:
data_missing_v=completing_missing_data(data_missing_v)

<ipython-input-18-61f9cfbee058>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Speed'] = data['Speed'].astype('float')


In [20]:
data_missing_v.count()

Speed             35436
RPM               35434
ENGINE_LOAD       35384
AmbientAirTemp    35424
ThrottlePos       35383
Angle_Rot         38708
dtype: int64

In [21]:
data[missing_values_list] = data_missing_v
data.count()

CIN               38710
altitude          38710
longitude         38710
Speed             35436
RPM               35434
ENGINE_LOAD       35384
AmbientAirTemp    35424
ThrottlePos       35383
X                 38710
Y                 38710
Z                 38710
zone              38710
time              38710
acc_angX          38710
acc_angY          38710
acc_angZ          38710
Angle_Rot         38708
dtype: int64

### remove any remaining missing data values

In [22]:
data = removing_incomplete_raws(data)

In [23]:
data.isnull().values.any() #checking for any null value in the data

False

In [24]:
data

,CIN,altitude,longitude,Speed,RPM,ENGINE_LOAD,AmbientAirTemp,ThrottlePos,X,Y,Z,zone,time,acc_angX,acc_angY,acc_angZ,Angle_Rot
0,6023226,34.729785,10.656346,43.00000,2112.000000,72.200000,42.000000,29.000000,0.083398,7.703288,5.959764,Urbaine,2020-07-22 17:12:20,0.490807,52.291513,37.742191,4.34468
1,6023226,34.729863,10.656757,52.00000,2559.000000,76.900000,42.000000,30.600000,2.218036,9.894187,2.721148,Urbaine,2020-07-22 17:12:24,12.202002,70.494356,15.028538,1.12742
2,6023226,34.730034,10.657389,61.00000,2974.000000,78.000000,42.000000,34.900000,0.567006,8.561572,4.537072,Urbaine,2020-07-22 17:12:28,3.350421,61.921423,27.880596,6.06838
3,6023226,34.730220,10.658135,65.00000,3173.000000,36.500000,41.000000,25.500000,-0.047080,8.618731,4.569990,Urbaine,2020-07-22 17:12:32,-0.276630,62.090683,27.945750,0.209587
4,6023226,34.730323,10.658877,62.00000,1443.000000,0.000000,41.000000,15.300000,5.573964,8.053125,4.679521,Urbaine,2020-07-22 17:12:36,30.911456,47.915213,25.549078,0.628612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35376,6023226,34.836305,10.763956,29.00000,2228.000000,24.700000,35.000000,18.800000,0.796000,7.788000,5.788000,Urbaine,2020-12-01 09:46:44,4.691684,53.145596,36.492526,2.1602
35377,6023226,34.835701,10.762994,24.74989,1956.871457,49.752625,34.984915,19.650579,-2.446000,4.255000,8.144000,Urbaine,2020-12-01 09:46:58,-14.912773,26.594099,58.949821,5.16187
35378,6023226,34.835565,10.762759,25.00000,2027.000000,46.866056,35.000000,20.219973,-0.672000,8.046000,4.811000,Urbaine,2020-12-01 09:47:02,-4.101840,58.903758,30.802133,0.782921
35379,6023226,34.835439,10.762487,28.00000,2204.000000,22.700000,35.000000,18.400000,-0.588000,4.618000,8.303000,Urbaine,2020-12-01 09:47:06,-3.542978,29.033565,60.747413,11.0593


In [25]:
data['CIN'] = data['CIN'].astype('float')
data.head(20)


,CIN,altitude,longitude,Speed,RPM,ENGINE_LOAD,AmbientAirTemp,ThrottlePos,X,Y,Z,zone,time,acc_angX,acc_angY,acc_angZ,Angle_Rot
0,6023226.0,34.729785,10.656346,43.0,2112.0,72.2,42.0,29.0,0.083398,7.703288,5.959764,Urbaine,2020-07-22 17:12:20,0.490807,52.291513,37.742191,4.34468
1,6023226.0,34.729863,10.656757,52.0,2559.0,76.9,42.0,30.6,2.218036,9.894187,2.721148,Urbaine,2020-07-22 17:12:24,12.202002,70.494356,15.028538,1.12742
2,6023226.0,34.730034,10.657389,61.0,2974.0,78.0,42.0,34.9,0.567006,8.561572,4.537072,Urbaine,2020-07-22 17:12:28,3.350421,61.921423,27.880596,6.06838
3,6023226.0,34.730220,10.658135,65.0,3173.0,36.5,41.0,25.5,-0.047080,8.618731,4.569990,Urbaine,2020-07-22 17:12:32,-0.276630,62.090683,27.945750,0.209587
4,6023226.0,34.730323,10.658877,62.0,1443.0,0.0,41.0,15.3,5.573964,8.053125,4.679521,Urbaine,2020-07-22 17:12:36,30.911456,47.915213,25.549078,0.628612
5,6023226.0,34.730427,10.659607,67.0,2348.0,92.2,41.0,49.0,5.629926,7.232548,3.689860,Urbaine,2020-07-22 17:12:40,34.751778,47.075472,21.938138,0.113447
6,6023226.0,34.730528,10.660373,63.0,2210.0,40.0,41.0,27.5,3.916350,8.944328,2.818408,Urbaine,2020-07-22 17:12:44,22.675787,61.681203,16.107501,0.875157
7,6023226.0,34.730638,10.661192,70.0,2450.0,94.1,41.0,27.8,6.156628,7.780797,3.455538,Urbaine,2020-07-22 17:12:48,35.887790,47.800442,19.209936,1.45026
8,6023226.0,34.730737,10.661857,59.0,2082.0,90.2,41.0,88.2,2.976367,8.937744,1.800318,Urbaine,2020-07-22 17:12:52,18.087129,68.763505,10.823935,2.02551
9,6023226.0,34.730825,10.662569,60.0,2122.0,57.3,41.0,17.6,6.170095,8.217421,4.684010,Urbaine,2020-07-22 17:12:56,33.131351,46.709000,24.514832,4.21092


### creating the route location column

In [26]:

target_conditions  = [ (data['CIN'] == 2),  (data['CIN'] == 20), (data['CIN'] == 10000002), (data['CIN'] == 2020202),(data['CIN'] == 3) , (data['CIN'] == 4),(data['CIN'] == 1), (data['CIN'] == 10), (data['CIN'] == 1010101),(data['CIN'] == 10000001), (data['CIN'] == 11067773)]
target_choices     = ['normal', 'normal', 'normal', 'normal','agressif','dangerous','timid','timid','timid','timid','timid']
data["TargetValue"] = np.select(target_conditions, target_choices, default='not specified')
data['CIN'] = data['CIN'].astype('int')

In [27]:
data['TargetValue'] = data['TargetValue'].astype('category')
data['zone'] = data['zone'].astype('category')

In [28]:
print(data['zone'].unique())
print(data['TargetValue'].unique())

['Urbaine', 'Nationale', 'Autoroute']
Categories (3, object): ['Urbaine', 'Nationale', 'Autoroute']
['not specified', 'timid', 'normal', 'agressif', 'dangerous']
Categories (5, object): ['not specified', 'timid', 'normal', 'agressif', 'dangerous']


### Rearange columns by putting time last

In [29]:
data = data.reindex(columns=(list([a for a in data.columns if a != 'time'])+ ['time']  ))#reorder columns by putting time last

In [30]:
data

,CIN,altitude,longitude,Speed,RPM,ENGINE_LOAD,AmbientAirTemp,ThrottlePos,X,Y,Z,zone,acc_angX,acc_angY,acc_angZ,Angle_Rot,TargetValue,time
0,6023226,34.729785,10.656346,43.00000,2112.000000,72.200000,42.000000,29.000000,0.083398,7.703288,5.959764,Urbaine,0.490807,52.291513,37.742191,4.34468,not specified,2020-07-22 17:12:20
1,6023226,34.729863,10.656757,52.00000,2559.000000,76.900000,42.000000,30.600000,2.218036,9.894187,2.721148,Urbaine,12.202002,70.494356,15.028538,1.12742,not specified,2020-07-22 17:12:24
2,6023226,34.730034,10.657389,61.00000,2974.000000,78.000000,42.000000,34.900000,0.567006,8.561572,4.537072,Urbaine,3.350421,61.921423,27.880596,6.06838,not specified,2020-07-22 17:12:28
3,6023226,34.730220,10.658135,65.00000,3173.000000,36.500000,41.000000,25.500000,-0.047080,8.618731,4.569990,Urbaine,-0.276630,62.090683,27.945750,0.209587,not specified,2020-07-22 17:12:32
4,6023226,34.730323,10.658877,62.00000,1443.000000,0.000000,41.000000,15.300000,5.573964,8.053125,4.679521,Urbaine,30.911456,47.915213,25.549078,0.628612,not specified,2020-07-22 17:12:36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35376,6023226,34.836305,10.763956,29.00000,2228.000000,24.700000,35.000000,18.800000,0.796000,7.788000,5.788000,Urbaine,4.691684,53.145596,36.492526,2.1602,not specified,2020-12-01 09:46:44
35377,6023226,34.835701,10.762994,24.74989,1956.871457,49.752625,34.984915,19.650579,-2.446000,4.255000,8.144000,Urbaine,-14.912773,26.594099,58.949821,5.16187,not specified,2020-12-01 09:46:58
35378,6023226,34.835565,10.762759,25.00000,2027.000000,46.866056,35.000000,20.219973,-0.672000,8.046000,4.811000,Urbaine,-4.101840,58.903758,30.802133,0.782921,not specified,2020-12-01 09:47:02
35379,6023226,34.835439,10.762487,28.00000,2204.000000,22.700000,35.000000,18.400000,-0.588000,4.618000,8.303000,Urbaine,-3.542978,29.033565,60.747413,11.0593,not specified,2020-12-01 09:47:06


### saving the shortcut for preprocessed data

In [31]:
df = data

In [32]:
#data.to_csv(r'fetched_serveur_data_preprocessed.csv', index=False)
#data.to_csv(r'local_database_preprocessed.csv', index=False)

In [33]:
df

,CIN,altitude,longitude,Speed,RPM,ENGINE_LOAD,AmbientAirTemp,ThrottlePos,X,Y,Z,zone,acc_angX,acc_angY,acc_angZ,Angle_Rot,TargetValue,time
0,6023226,34.729785,10.656346,43.00000,2112.000000,72.200000,42.000000,29.000000,0.083398,7.703288,5.959764,Urbaine,0.490807,52.291513,37.742191,4.34468,not specified,2020-07-22 17:12:20
1,6023226,34.729863,10.656757,52.00000,2559.000000,76.900000,42.000000,30.600000,2.218036,9.894187,2.721148,Urbaine,12.202002,70.494356,15.028538,1.12742,not specified,2020-07-22 17:12:24
2,6023226,34.730034,10.657389,61.00000,2974.000000,78.000000,42.000000,34.900000,0.567006,8.561572,4.537072,Urbaine,3.350421,61.921423,27.880596,6.06838,not specified,2020-07-22 17:12:28
3,6023226,34.730220,10.658135,65.00000,3173.000000,36.500000,41.000000,25.500000,-0.047080,8.618731,4.569990,Urbaine,-0.276630,62.090683,27.945750,0.209587,not specified,2020-07-22 17:12:32
4,6023226,34.730323,10.658877,62.00000,1443.000000,0.000000,41.000000,15.300000,5.573964,8.053125,4.679521,Urbaine,30.911456,47.915213,25.549078,0.628612,not specified,2020-07-22 17:12:36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35376,6023226,34.836305,10.763956,29.00000,2228.000000,24.700000,35.000000,18.800000,0.796000,7.788000,5.788000,Urbaine,4.691684,53.145596,36.492526,2.1602,not specified,2020-12-01 09:46:44
35377,6023226,34.835701,10.762994,24.74989,1956.871457,49.752625,34.984915,19.650579,-2.446000,4.255000,8.144000,Urbaine,-14.912773,26.594099,58.949821,5.16187,not specified,2020-12-01 09:46:58
35378,6023226,34.835565,10.762759,25.00000,2027.000000,46.866056,35.000000,20.219973,-0.672000,8.046000,4.811000,Urbaine,-4.101840,58.903758,30.802133,0.782921,not specified,2020-12-01 09:47:02
35379,6023226,34.835439,10.762487,28.00000,2204.000000,22.700000,35.000000,18.400000,-0.588000,4.618000,8.303000,Urbaine,-3.542978,29.033565,60.747413,11.0593,not specified,2020-12-01 09:47:06


In [34]:
def Time_conv(time): # time est sous le format "jour/mois/année h:min"
    l=str(time).split(' ')  # time extraction
    time =l[-1].split(':') # extraire h et min
    time_converted=int(time[0])*3600+int(time[1])*60 + int(time[2]) # convertir en  sec
    #print(time[0], time[1], time[2])
    
    return(time_converted)

def time_conv_list(trip):
    l=list()
    for temps in trip['time']:
        #print("time conv", Time_conv(temps))
        l.append(Time_conv(temps)) # enregistrer les temps convertis dans une liste
    return(l)

def trip_segmentation(trip):
    l=list()
    for index in trip['time']:
        l.append(Time_conv(index)) # enregistrer les temps convertis dans une liste
    time_limits=list()
    time_limits.append(0)
    for index in range(len(l)-1): # parcourir la liste pour segmenter les trips
       
        if ((l[index+1]-l[index])>20): # si la difference en time entre deux valeurs successives depasse une min on sépare les trips
            time_limits.append(index)
            time_limits.append(index+1)# liste des indexs des différents trips
    time_limits.append(len(trip)-1)
    print("time trip limits : \n", time_limits)
    new_time_limits=list()
    w=(len(time_limits))
    for i in range(0,(w-1),2):

        if (time_limits[i+1]-time_limits[i] < 10): # si le trip contient mois que 10 valeurs on l élimine
            continue
        new_time_limits.append(time_limits[i])
        new_time_limits.append(time_limits[i+1])
    return(new_time_limits)




In [35]:
print(data['time'][1])
print(Time_conv(data['time'][1]))

2020-07-22 17:12:24
61944


In [36]:
Time_conv(data['time'][1])

61944

In [37]:
l=time_conv_list(data)
l
#print("liste des temps convertis : \n", l)

[61940,
 61944,
 61948,
 61952,
 61956,
 61960,
 61964,
 61968,
 61972,
 61976,
 61980,
 61984,
 61988,
 61992,
 61996,
 62000,
 62004,
 62008,
 62012,
 62016,
 62020,
 62024,
 62028,
 62032,
 62036,
 62040,
 62044,
 62048,
 62052,
 62056,
 62060,
 62064,
 62068,
 62072,
 62076,
 62080,
 62084,
 62088,
 62092,
 62096,
 62100,
 62104,
 62108,
 62112,
 62116,
 62120,
 62124,
 62128,
 62131,
 62134,
 62138,
 62142,
 62146,
 62149,
 62153,
 62157,
 62161,
 62165,
 62169,
 62173,
 62177,
 62181,
 62185,
 62189,
 62193,
 62197,
 62201,
 62205,
 62209,
 62213,
 62217,
 62221,
 62225,
 62229,
 62233,
 62237,
 62241,
 62245,
 62248,
 62252,
 62256,
 62260,
 62264,
 62268,
 62272,
 62276,
 62280,
 62284,
 62288,
 62292,
 62296,
 62300,
 62304,
 66760,
 66764,
 66768,
 70066,
 70070,
 70074,
 70078,
 70082,
 70086,
 70090,
 70094,
 70098,
 70102,
 70106,
 70110,
 70114,
 70118,
 70122,
 70126,
 70130,
 70134,
 70138,
 70142,
 70146,
 70150,
 70154,
 70176,
 70180,
 70184,
 70188,
 70203,
 70207,


In [38]:
df['CIN'].unique()

array([ 6023226,  1010101,        2,        3,        4,        1,
       10000001, 10000002,       10,       20,  1234567, 11067773])

In [39]:

group = df.groupby('CIN')

normal_df = group.get_group(2)
agressif_df = group.get_group(3)
dangerous_df = group.get_group(4)
timid_df = group.get_group(1)
timid_df = timid_df.reset_index(drop=True)
normal_df =normal_df.reset_index(drop=True)
agressif_df = agressif_df.reset_index(drop=True)
dangerous_df =dangerous_df.reset_index(drop=True)


In [40]:
#segs=trip_segmentation(data)
segs_normal=trip_segmentation(normal_df)
segs_agressif=trip_segmentation(agressif_df)
segs_dangerous=trip_segmentation(dangerous_df)
segs_timid=trip_segmentation(timid_df)

time trip limits : 
 [0, 148, 149, 196, 197, 342, 343, 637, 638, 741, 742, 860, 861, 863, 864, 904, 905, 907, 908, 962, 963, 1107, 1108, 1352, 1353, 1503, 1504, 1627, 1628, 1628, 1629, 1634, 1635, 1917, 1918, 1958, 1959, 1961, 1962, 1993, 1994, 2019, 2020, 2049, 2050, 2324, 2325, 2327, 2328, 2816, 2817, 2817, 2818, 3069, 3070, 3239, 3240, 3333, 3334, 3785, 3786, 3788, 3789, 3858, 3859, 3878, 3879, 3886, 3887, 3966, 3967, 4028, 4029, 4169, 4170, 4172, 4173, 4240, 4241, 4478, 4479, 4481, 4482, 4559, 4560, 4828, 4829, 4832, 4833, 5156, 5157, 5206, 5207, 5340, 5341, 5440, 5441, 5443, 5444, 5734, 5735, 5800, 5801, 5885, 5886, 5955, 5956, 5958, 5959, 5980, 5981, 6131, 6132, 6281, 6282, 6284, 6285, 6692, 6693, 6730, 6731, 7686, 7687, 7689, 7690, 7752, 7753, 7755, 7756, 8119, 8120, 8445, 8446, 8872, 8873, 8961, 8962, 8969, 8970, 8995, 8996, 9021]
time trip limits : 
 [0, 158, 159, 176, 177, 298, 299, 548, 549, 692, 693, 708, 709, 896, 897, 984, 985, 987, 988, 1015, 1016, 1028, 1029, 1110, 1111

In [41]:
# eliminer les fausses trips de la liste des trips initiale
def del_unsignicant_trip(data, segs):
    trip_cleaned=data.copy(deep='False')
    #print(len(trip_cleaned))
    x =tuple()
    segments=list()
    for i in range(0,len(segs)-1,2):
        x=(segs[i], segs[i+1])
        segments.append(x) 
    print(segments)

    for i in range(len(segments)-1):
        if segments[i+1][0] - segments[i][1]==1:
            print("dropped segs", segments[i], segments[i+1])
            continue
        print(range(segments[i][1]+1, segments[i+1][0] ))
        print(trip_cleaned)
        trip_cleaned = trip_cleaned.drop(index=range(segments[i][1]+1, segments[i+1][0] ), inplace=False)
    trip_cleaned = trip_cleaned.drop(index=range(len(trip_cleaned)-(len(trip_cleaned))%10-1, len(trip_cleaned)-1 ), inplace=False)
        
    return(trip_cleaned)

In [42]:
normal_trip_cleaned=del_unsignicant_trip(normal_df, segs_normal)
#print(normal_trip_cleaned)
agressif_trip_cleaned=del_unsignicant_trip(agressif_df, segs_agressif)
dangerous_trip_cleaned=del_unsignicant_trip(dangerous_df, segs_dangerous)
timid_trip_cleaned=del_unsignicant_trip(timid_df, segs_timid)

[(0, 148), (149, 196), (197, 342), (343, 637), (638, 741), (742, 860), (864, 904), (908, 962), (963, 1107), (1108, 1352), (1353, 1503), (1504, 1627), (1635, 1917), (1918, 1958), (1962, 1993), (1994, 2019), (2020, 2049), (2050, 2324), (2328, 2816), (2818, 3069), (3070, 3239), (3240, 3333), (3334, 3785), (3789, 3858), (3859, 3878), (3887, 3966), (3967, 4028), (4029, 4169), (4173, 4240), (4241, 4478), (4482, 4559), (4560, 4828), (4833, 5156), (5157, 5206), (5207, 5340), (5341, 5440), (5444, 5734), (5735, 5800), (5801, 5885), (5886, 5955), (5959, 5980), (5981, 6131), (6132, 6281), (6285, 6692), (6693, 6730), (6731, 7686), (7690, 7752), (7756, 8119), (8120, 8445), (8446, 8872), (8873, 8961), (8970, 8995), (8996, 9021)]
dropped segs (0, 148) (149, 196)
dropped segs (149, 196) (197, 342)
dropped segs (197, 342) (343, 637)
dropped segs (343, 637) (638, 741)
dropped segs (638, 741) (742, 860)
range(861, 864)
      CIN   altitude  longitude  Speed     RPM  ENGINE_LOAD  AmbientAirTemp  \
0       

dropped segs (8970, 8995) (8996, 9021)
[(0, 158), (159, 176), (177, 298), (299, 548), (549, 692), (693, 708), (709, 896), (897, 984), (988, 1015), (1016, 1028), (1029, 1110), (1113, 1236), (1237, 1265), (1269, 1333), (1337, 1365), (1366, 1427), (1428, 1504), (1505, 1604), (1605, 1742), (1743, 1833), (1834, 1964), (1965, 2019), (2020, 2246), (2247, 2290), (2291, 2364), (2369, 2435), (2436, 2557), (2558, 2712), (2716, 2938), (2939, 3036), (3037, 3188), (3189, 3212), (3213, 3351), (3352, 3677), (3678, 3939), (3940, 4082), (4088, 4174), (4178, 4328), (4329, 4358), (4359, 4731), (4735, 4863), (4867, 5027), (5028, 5134), (5135, 5181), (5182, 5315), (5316, 5512), (5513, 5570), (5571, 5700), (5701, 5852), (5853, 6060), (6061, 6336), (6340, 6493), (6494, 6604), (6605, 6843), (6844, 7118), (7119, 7148), (7149, 7221), (7240, 7312)]
dropped segs (0, 158) (159, 176)
dropped segs (159, 176) (177, 298)
dropped segs (177, 298) (299, 548)
dropped segs (299, 548) (549, 692)
dropped segs (549, 692) (693,

dropped segs (840, 862) (863, 879)
dropped segs (863, 879) (880, 951)
dropped segs (880, 951) (952, 1206)
dropped segs (952, 1206) (1207, 1611)
dropped segs (1207, 1611) (1612, 1703)
range(1704, 1706)
      CIN   altitude  longitude      Speed          RPM  ENGINE_LOAD  \
0       1  34.835868  10.754720  -2.214375   795.393858     8.114249   
1       1  34.835877  10.754786  -1.321462   326.762082     4.252770   
2       1  34.835866  10.754799   0.000000     0.000000     1.889906   
3       1  34.835865  10.754808   0.000000     0.000000     0.000000   
4       1  34.835864  10.754808   0.000000     0.000000     0.000000   
...   ...        ...        ...        ...          ...          ...   
3238    1  34.732028  10.710480  33.000000  1622.000000    67.100000   
3239    1  34.731818  10.710876  41.000000  1989.000000    43.900000   
3240    1  34.731595  10.711282  40.000000  1973.000000     0.000000   
3241    1  34.731302  10.711633  39.000000  1898.000000    20.000000   
3242   

In [43]:
print(len(dangerous_trip_cleaned), len(agressif_trip_cleaned), len(timid_trip_cleaned), len(normal_trip_cleaned))

1180 7270 3200 8950


In [44]:
trip_cleaned = pd.concat([timid_trip_cleaned, normal_trip_cleaned, agressif_trip_cleaned, dangerous_trip_cleaned], sort=False)

# Classification

### Load preprocessed data shortcut file 

In [45]:
len(trip_cleaned)

20600

In [46]:
trip_cleaned.head()

,CIN,altitude,longitude,Speed,RPM,ENGINE_LOAD,AmbientAirTemp,ThrottlePos,X,Y,Z,zone,acc_angX,acc_angY,acc_angZ,Angle_Rot,TargetValue,time
0,1,34.835868,10.754720,-2.214375,795.393858,8.114249,49.0,12.646402,1.894,2.322,9.991,Urbaine,10.466214,12.867968,73.336034,46.7268,timid,2020-08-24 10:33:58
1,1,34.835877,10.754786,-1.321462,326.762082,4.252770,49.0,13.277184,0.141,3.312,9.252,Urbaine,0.822388,19.702396,70.317064,29.7964,timid,2020-08-24 10:34:02
2,1,34.835866,10.754799,0.000000,0.000000,1.889906,49.0,15.019374,0.069,3.291,9.238,Urbaine,0.403297,19.616018,70.417539,73.3185,timid,2020-08-24 10:34:06
3,1,34.835865,10.754808,0.000000,0.000000,0.000000,49.0,16.500000,0.077,3.320,9.221,Urbaine,0.450340,19.809030,70.223465,127.811,timid,2020-08-24 10:34:10
4,1,34.835864,10.754808,0.000000,0.000000,0.000000,49.0,16.500000,0.080,3.323,9.211,Urbaine,0.468286,19.845330,70.186724,43.5679,timid,2020-08-24 10:34:14


In [47]:
trip_cleaned.to_csv(r'local_database_preprocessed.csv', index=False)

In [48]:
data = pd.read_csv('local_database_preprocessed.csv')


In [49]:
data
#trip_cleaned = data(index=range(segments[][1]+1, segments[i+1][0] ), inplace=False)

,CIN,altitude,longitude,Speed,RPM,ENGINE_LOAD,AmbientAirTemp,ThrottlePos,X,Y,Z,zone,acc_angX,acc_angY,acc_angZ,Angle_Rot,TargetValue,time
0,1,34.835868,10.754720,-2.214375,795.393858,8.114249,49.0,12.646402,1.894,2.322,9.991,Urbaine,10.466214,12.867968,73.336034,46.726764,timid,2020-08-24 10:33:58
1,1,34.835877,10.754786,-1.321462,326.762082,4.252770,49.0,13.277184,0.141,3.312,9.252,Urbaine,0.822388,19.702396,70.317064,29.796423,timid,2020-08-24 10:34:02
2,1,34.835866,10.754799,0.000000,0.000000,1.889906,49.0,15.019374,0.069,3.291,9.238,Urbaine,0.403297,19.616018,70.417539,73.318479,timid,2020-08-24 10:34:06
3,1,34.835865,10.754808,0.000000,0.000000,0.000000,49.0,16.500000,0.077,3.320,9.221,Urbaine,0.450340,19.809030,70.223465,127.811463,timid,2020-08-24 10:34:10
4,1,34.835864,10.754808,0.000000,0.000000,0.000000,49.0,16.500000,0.080,3.323,9.211,Urbaine,0.468286,19.845330,70.186724,43.567904,timid,2020-08-24 10:34:14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20595,4,34.837743,10.755090,22.000000,914.000000,8.200000,26.0,14.900000,-0.328,1.238,9.944,Urbaine,-1.875530,7.095829,82.696069,24.993138,dangerous,2020-11-21 08:26:23
20596,4,34.837703,10.755176,9.000000,1523.000000,26.300000,26.0,20.000000,-1.637,2.546,9.315,Urbaine,-9.625385,15.073068,72.029136,102.662401,dangerous,2020-11-21 08:26:27
20597,4,34.837703,10.755217,5.000000,980.000000,12.900000,26.0,14.100000,0.476,2.635,9.393,Urbaine,2.794570,15.657817,74.120138,105.142333,dangerous,2020-11-21 08:26:31
20598,4,34.837698,10.755216,0.000000,845.000000,12.900000,26.0,14.500000,3.179,0.831,9.574,Urbaine,18.312023,4.711109,71.087702,135.485031,dangerous,2020-11-21 08:26:35


In [51]:
import os

os.system('mkdir user1')
os.system('mkdir user2')
os.system('mkdir user3')
os.system('mkdir user4')

new_df=data.assign(trip_id=0)
bibliotheque = pd.DataFrame(columns=['CIN','trip_id','Date'])
dep=-1
arr=0
x1=0
x2=0
x3=0
x4=0
x5=0
j=1
for i in range(len(new_df)-1):
      if((Time_conv(new_df['time'] [i+1])-Time_conv(new_df['time'] [i])>30) or(new_df['CIN'] [i+1]!=new_df['CIN'] [i])):
            dep=arr
            arr=i
            #print((dep,arr))
            
            new=new_df[dep+1:arr+1]
            
            if(new['CIN'][dep+1]==1):
                
                x1+=1
                bibliotheque.loc[j]=[ new['CIN'][dep+1], x1, new['time'][dep+1] ]
                j+=1
                os.system('echo.> user1/trip'+str(x1)+'.csv')
                new1=new
                new1.drop(["CIN", "altitude","ENGINE_LOAD","ThrottlePos","zone","TargetValue","time","trip_id"], axis=1)
                new1.to_csv('user1/trip'+str(x1)+'.csv', index=False)
            elif(new['CIN'][dep+1]==2):
                x2+=1
                bibliotheque.loc[j]=[ new['CIN'][dep+1], x2, new['time'][dep+1] ]
                j+=1 
                new1=new
                new1.drop(["CIN", "altitude","ENGINE_LOAD","ThrottlePos","zone","TargetValue","time","trip_id"], axis=1)
                os.system('echo.> user2/trip'+str(x1)+'.csv')
                new1.to_csv('user2/trip'+str(x2)+'.csv', index=False)
            
            elif(new['CIN'][dep+1]==3):
                x3+=1
                bibliotheque.loc[j]=[ new['CIN'][dep+1], x3, new['time'][dep+1] ]
                j+=1
                new1=new
                new1.drop(["CIN", "altitude","ENGINE_LOAD","ThrottlePos","zone","TargetValue","time","trip_id"], axis=1)
                os.system('echo.> user3/trip'+str(x1)+'.csv')
                new1.to_csv('user3/trip'+str(x3)+'.csv', index=False)  
            elif(new['CIN'][dep+1]==4):
                x4+=1
                bibliotheque.loc[j]=[ new['CIN'][dep+1], x4, new['time'][dep+1] ]
                j+=1 
                new1=new
                new1.drop(["CIN", "altitude","ENGINE_LOAD","ThrottlePos","zone","TargetValue","time","trip_id"], axis=1)
                os.system('echo.> user4/trip'+str(x1)+'.csv')
                new1.to_csv('user4/trip'+str(x4)+'.csv', index=False)
            
            

bibliotheque.head(100)
os.system('echo.> triptable.csv')
bibliotheque.to_csv('triptable.csv', index=False)









